# Support Vector Machines

Leitura dos datasets tratados

In [1]:
import sklearn as skl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

allowed_nans = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN',
                '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NA',
                'NULL', 'NaN', 'n/a', 'nan', 'null']

treino = pd.read_csv('datasets/treinoTratado.csv', na_values=allowed_nans, keep_default_na=False)
teste = pd.read_csv('datasets/testeTratado.csv', na_values=allowed_nans, keep_default_na=False)

## Construção e Modelação

In [2]:
label_encoder = preprocessing.LabelEncoder() 

treino['Date']= label_encoder.fit_transform(treino['Date'])
teste['Date']= label_encoder.fit_transform(teste['Date'])

treino['Month']= label_encoder.fit_transform(treino['Month'])
teste['Month']= label_encoder.fit_transform(teste['Month'])

treino['Season']= label_encoder.fit_transform(treino['Season'])
teste['Season']= label_encoder.fit_transform(teste['Season'])

treino['weather_description']= label_encoder.fit_transform(treino['weather_description'])
teste['weather_description']= label_encoder.fit_transform(teste['weather_description'])

In [3]:
X_treino = treino.drop('Injeçao na rede (kWh)', axis=1)
y_treino = treino['Injeçao na rede (kWh)']

X_teste = teste

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm

svm_model = svm.SVC(random_state=2023)

# gamma: [1, 0.1, 0.01, 0.001, 0.0001]

params = {
    'C': [0.1, 1, 10, 100, 100], 'gamma': [1, 0.1, 0.01], 'kernel': ['rbf']
}

grid = GridSearchCV(estimator=svm_model,
                    param_grid=params,
                    cv=5,
                    n_jobs=1,
                    verbose=2)

grid.fit(X_treino, y_treino)
grid.best_score_, grid.best_params_

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  10.7s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  10.6s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  11.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  12.7s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  12.6s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  12.3s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  11.1s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  10.8s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  11.3s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  11.4s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   6.7s
[CV] END ......................C=0.1, gamma=0.01

(0.705973128541924, {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'})

In [5]:
predictions = grid.predict(X_teste)

predict_data = {'Result': predictions}
df = pd.DataFrame(predict_data)
df.index += 1
df.index.name = 'RowId'

df.to_csv(f'predictions/svmPrediction.csv')